In [6]:
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [ ]:
data=pd.read_csv("Churn_Modelling.csv")

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd

# Drop unnecessary columns
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)

# Encode Gender (Label Encoding)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

# Encode Geography (One-Hot Encoding with drop to avoid dummy trap)
onehot_encoder_geo = OneHotEncoder(sparse=False, drop='first')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

# Merge back
data = pd.concat([data.drop('Geography', axis=1).reset_index(drop=True), geo_encoded_df.reset_index(drop=True)], axis=1)

In [8]:
X=data.drop('Exited',axis=1)
y=data['Exited']

## split the dataset into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


## scaling these features

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [9]:
## save the encoder or scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [10]:
# define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[0],)))

    for _ in range(layers-1):
        model.add (Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [14]:
##create a keras Classifier

model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [15]:
# define the grid search parameters
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2,3],
    'epochs':[50,100]
}

In [16]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(X_train,y_train)

/Users/vijaykumar/Desktop/DeepLearning/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/vijaykumar/Desktop/DeepLearning/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/vijaykumar/Desktop/DeepLearning/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/vijaykumar/Desktop/DeepLearning/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which poi

ValueError: 
All the 72 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/vijaykumar/Desktop/DeepLearning/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/vijaykumar/Desktop/DeepLearning/venv/lib/python3.11/site-packages/scikeras/wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/vijaykumar/Desktop/DeepLearning/venv/lib/python3.11/site-packages/scikeras/wrappers.py", line 760, in fit
    self._fit(
  File "/Users/vijaykumar/Desktop/DeepLearning/venv/lib/python3.11/site-packages/scikeras/wrappers.py", line 928, in _fit
   

In [ ]:
grid_result.best_score_

In [ ]:
grid_result.best_params_